<a href="https://colab.research.google.com/github/RUMONMD89/Computing-science-with-Fortran/blob/main/ACS8REV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/FORTRAN

Mounted at /content/drive
/content/drive/MyDrive/FORTRAN


In [2]:
%%writefile ACS8.f03
! _/_/_/ Jacobi/SOR _/_/_/
! 2022.08.19 Written  by Y.Hirokawa
! 2022.11.10 Modified by Y.Hirokawa
! 2023.11.21 Modified by Y.Hirokawa

program main
  implicit none
  integer, parameter          :: NDIM=3, NMAX=2000, IDEBUG=0, ISEED=1
  double precision, parameter :: THRES=1.0d-4
  double precision            :: a(NDIM,NDIM), b(NDIM), x(NDIM), ths, w
  integer                     :: i, j

  ! Initialize
  call sub_init(a,b,x,NDIM,IDEBUG)

  ! Output Data
  write(*,*) "=== Initial Value ==="
  call sub_output(a,b,x,NDIM)

  ! Output value of threshold
  write(*,*) ""
  write(*,*) "Threshold =", THRES

  ! Jacobi
  write(*,*) ""
  write(*,*) ""
  write(*,*) "=== Jacobi ==="
  call sub_jacobi(a,b,x,NDIM,NMAX,THRES,W)
  call sub_output(a,b,x,NDIM)

  ! Gauss Seidel
  write(*,*) ""
  write(*,*) ""
  write(*,*) "=== Gauss Seidel ==="
  w = 1.0
  call sub_init(a,b,x,NDIM,IDEBUG)
  call sub_sor(a,b,x,NDIM,NMAX,THRES,W)
  call sub_output(a,b,x,NDIM)

  ! SOR
  write(*,*) ""
  write(*,*) ""
  write(*,*) "=== Sccessive Over Relaxation (w=1.1) ==="
  w = 1.1
  call sub_init(a,b,x,NDIM,IDEBUG)
  call sub_sor(a,b,x,NDIM,NMAX,THRES,W)
  call sub_output(a,b,x,NDIM)

  ! Gradient Descent
  write(*,*) ""
  write(*,*) ""
  write(*,*) "=== Gradient Descent (Steepest Descent) ==="
  call sub_init(a,b,x,NDIM,IDEBUG)
  call sub_gd(a,b,x,NDIM,NMAX,THRES,W)
  call sub_output(a,b,x,NDIM)

  ! Conjugate Gradient
  write(*,*) ""
  write(*,*) ""
  write(*,*) "=== Conjugate Gradient ==="
  call sub_init(a,b,x,NDIM,IDEBUG)
  call sub_cg(a,b,x,NDIM,NMAX,THRES,W)
  call sub_output(a,b,x,NDIM)

  stop


contains


  ! Initialize variables
  subroutine sub_init(a,b,x,NDIM,IDEBUG)
    integer, intent(in)             :: NDIM, IDEBUG
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)
    double precision                :: tmp
    integer                         :: nseed
    integer, allocatable            :: seed(:)

    ! Initialize Puseudo Random Number
    call random_seed(size=nseed)
    allocate(seed(nseed))
    seed(:) = ISEED
    call random_seed(put=seed)

    ! Initialization
    do j = 1, NDIM
      do i = 1, NDIM
         call random_number(tmp)
         if(i == j) then
           a(j,i) = dble(int(10.0*tmp))
         else
           a(j,i) = 1.0
         endif
      enddo
      call random_number(tmp)
      b(j) = dble(int(10.0*tmp))
      x(j) = 0.0
    enddo

    deallocate(seed)

    ! Debug data set (For use, please set IDEBUG=1 and NDIM=3 in parameter)
    if(IDEBUG == 1 .and. NDIM == 3) then
      a(1,1) = 2.0
      a(1,2) = 1.0
      a(1,3) = 2.0
      a(2,1) = 1.0
      a(2,2) = 3.0
      a(2,3) = 1.0
      a(3,1) = 1.0
      a(3,2) = 1.0
      a(3,3) = 3.0
      b(1) = 5.0
      b(2) = 0.0
      b(3) = 6.0
    endif

    return
  end subroutine sub_init


  ! Jacobi method
  subroutine sub_jacobi(a,b,x,NDIM,NMAX,THRES,W)
    integer, intent(in)             :: NDIM, NMAX
    double precision, intent(in)    :: THRES, W
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)
    integer                         :: i, n
    double precision                :: c, dm, tmp, res(NDIM), res_sum, x_old(NDIM), xn

    ! Iteration
    do n = 1, NMAX
      ! Backup Data
      x_old(:) = x(:)

      ! Select a Variable
      do j = 1, NDIM
        tmp = b(j)
        do i = 1, j-1
          tmp = tmp - a(j,i)*x_old(i)
        enddo
        do i = j+1, NDIM
          tmp = tmp - a(j,i)*x_old(i)
        enddo
        xn   = tmp/a(j,j)
        x(j) = xn
      enddo

      ! Calculate Difference
      res(:)  = x(:) - x_old(:)
      res(:)  = abs(res(:))
      res_sum = sum(res)

      ! Check Residuals
      write(*,*) "iteration=",n,"residuals=",res_sum
      if(res_sum <= THRES) then
        exit
      endif
    enddo

    return
  end subroutine sub_jacobi


  ! Gauss Seidel for w=1.0 / Successive Over Relaxation
  subroutine sub_sor(a,b,x,NDIM,NMAX,THRES,W)
    integer, intent(in)             :: NDIM, NMAX
    double precision, intent(in)    :: THRES, W
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)
    integer                         :: i, n
    double precision                :: c, dm, tmp, res(NDIM), res_sum, x_old(NDIM), xn

    ! Iteration
    do n = 1, NMAX
      ! Backup Data
      x_old(:) = x(:)

      ! Select a Variable
      do j = 1, NDIM
        tmp = b(j)
        do i = 1, j-1
          tmp = tmp - a(j,i)*x(i)
        enddo
        do i = j+1, NDIM
          tmp = tmp - a(j,i)*x(i)
        enddo
        xn   = tmp/a(j,j)
         x(j) = (1.0 - W)*x_old(j) + W*xn
      enddo

      ! Calculate Difference
      res(:)  = x(:) - x_old(:)
      res(:)  = abs(res(:))
      res_sum = sum(res)

      ! Check Residuals
      write(*,*) "iteration=",n,"residuals=",res_sum
      if(res_sum <= THRES) then
        exit
      endif
    enddo

    return
  end subroutine sub_sor


  ! Grand Descent Method (Steepest Descent Method)
  subroutine sub_gd(a,b,x,NDIM,NMAX,THRES,W)
    integer, intent(in)             :: NDIM, NMAX
    double precision, intent(in)    :: THRES, W
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)
    integer                         :: i, j, n
    double precision                :: alpha, beta, res_sum, tmp, r(NDIM), r_old(NDIM), p(NDIM)

    ! Initialiaztion: calculate p = b - Ax
    do j = 1, NDIM
      tmp = 0.0
      do i = 1, NDIM
        tmp = tmp + A(j,i)*x(i)
      enddo
      p(j) = b(j) - tmp
    enddo

    ! Iteration
    do n = 1, NMAX

      ! Check Residuals
      res_sum = sum(abs(p))
      write(*,*) "iteration=",n,"residuals=",res_sum

      if(res_sum <= THRES) then
        exit
      endif

      ! Calculate correction amount: alpha = (p,p)/(p,Ap)
      alpha = 0.0
      tmp   = 0.0
      do j = 1, NDIM
        alpha = alpha + p(j)*p(j)
        do i = 1, NDIM
          tmp   = tmp + p(j)*A(j,i)*p(i)
        enddo
      enddo
      alpha = alpha/tmp

      ! Update solution: x_new = x + alpha*p
      do j = 1, NDIM
        x(j) = x(j) + alpha*p(j)
      enddo

      ! Update correction vector: p = b - Ax
      do j = 1, NDIM
        tmp = 0.0
        do i = 1, NDIM
          tmp = tmp + A(j,i)*x(i)
        enddo
        p(j) = b(j) - tmp
      enddo
    enddo

    return
  end subroutine sub_gd


  ! Conjugate Gradient
  subroutine sub_cg(a,b,x,NDIM,NMAX,THRES,W)
    integer, intent(in)             :: NDIM, NMAX
    double precision, intent(in)    :: THRES, W
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)
    integer                         :: i, j, n
    double precision                :: alpha, beta, res_sum, tmp, r(NDIM), r_old(NDIM), p(NDIM)
    double precision                :: bnorm, rnorm

    ! Initialization: Calculate r = b - Ax and set p = r
    do j = 1, NDIM
      tmp = 0.0
      do i = 1, NDIM
        tmp = tmp + A(j,i)*x(i)
      enddo
      r(j) = b(j) - tmp
      p(j) = r(j)
    enddo

    ! Iteration
    do n = 1, NMAX
      ! Backup residual vector
      r_old(:) = r(:)

      ! Calculate correction amount: alpha = (p,r)/(p,Ap)
      alpha = 0.0
      tmp   = 0.0
      do j = 1, NDIM
        alpha = alpha + p(j)*r(j)
        do i = 1, NDIM
          tmp   = tmp + p(j)*A(j,i)*p(i)
        enddo
      enddo
      alpha = alpha/tmp

      ! Update solution and residual vector: x_new = x + alpha*p and r = r - alpha*Ap
      do j = 1, NDIM
        x(j) = x(j) + alpha*p(j)
        do i = 1, NDIM
          r(j) = r(j) - alpha*A(j,i)*p(i)
        enddo
      enddo

      ! Check Residuals
      bnorm = 0
      rnorm = 0
      do j = 1, NDIM
        bnorm = bnorm + b(j)*b(j)
        rnorm = rnorm + r(j)*r(j)
      enddo
      bnorm = sqrt(bnorm)
      rnorm = sqrt(rnorm)
      res_sum = rnorm/bnorm
      write(*,*) "iteration=",n,"residuals=",res_sum

      if(res_sum <= THRES) then
        exit
      endif

      ! Calculate parameter where p_new and p are othogonal (conjugate): beta = (r,r)/(r_old,r_old)
      beta = 0.0
      tmp  = 0.0
      do j = 1, NDIM
        beta = beta + r(j)*r(j)
        tmp  = tmp  + r_old(j)*r_old(j)
      enddo
      beta = beta/tmp

      ! Update correction vector: p = r + beta*p
      do j = 1, NDIM
        p(j) = r(j) + beta*p(j)
      enddo
    enddo

    return
  end subroutine sub_cg


  ! Output
  subroutine sub_output(a,b,x,NDIM)
    integer, intent(in)             :: NDIM
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), x(NDIM)

    write(*,*) "A="
    do j = 1, NDIM
      do i = 1, NDIM
        write(*,'(1f5.1)',advance='no') a(j,i)
      enddo
      write(*,*) ""
    enddo

    write(*,*) "B="
    do j = 1, NDIM
      write(*,'(1f5.1)') b(j)
    enddo

    write(*,*) "X="
    do j = 1, NDIM
      write(*,*) x(j)
    enddo

    return
  end subroutine sub_output

end program main

Overwriting ACS8.f03


In [3]:
DEBUG=0
EXE="ACS8"
if DEBUG == 0:
  !gfortran {EXE}.f03 -o ./{EXE}.out
  !./{EXE}.out
else:
  !sudo apt install gdb
  !gfortran {EXE}.f03 -o ./{EXE}.out -g
  !echo "run" | gdb ./{EXE}.out

 === Initial Value ===
 A=
  4.0  1.0  1.0 
  1.0  4.0  1.0 
  1.0  1.0  1.0 
 B=
  3.0
  5.0
  3.0
 X=
   0.0000000000000000     
   0.0000000000000000     
   0.0000000000000000     
 
 Threshold =   1.0000000000000000E-004
 
 
 === Jacobi ===
 iteration=           1 residuals=   5.0000000000000000     
 iteration=           2 residuals=   4.0000000000000000     
 iteration=           3 residuals=   3.5000000000000000     
 iteration=           4 residuals=   2.8750000000000000     
 iteration=           5 residuals=   2.4687500000000000     
 iteration=           6 residuals=   2.0546875000000000     
 iteration=           7 residuals=   1.7480468750000000     
 iteration=           8 residuals=   1.4643554687500000     
 iteration=           9 residuals=   1.2401123046875000     
 iteration=          10 residuals=   1.0422058105468750     
 iteration=          11 residuals=  0.88060760498046875     
 iteration=          12 residuals=  0.74125480651855469     
 iteration=          1